In [ ]:
import numpy as np
import pandas as pd
import scipy as sc
import sklearn as sk
import obspy as obs
from obspy.clients.fdsn import Client
from obspy.core import UTCDateTime
import pickle
import time
import matplotlib.pyplot as plt

In [ ]:
# Parameters
minYear = 2000 # filters both volcanoes and earthquakes
minMagnitude = 5.5 # m5.5 is the smallest mag
minMagnitude_query = 4
maxRadius = 0.8 # 1 degree of latitude/longitude is never more than ~111km. Search radius around a queried event
HoursWindow_before = 24 # Number of hours before and after an event to search for related events
HoursWindow_after = 24
maxDepthWindow = 10000 # Only search for events within this many meters depth of the primary event


In [ ]:
earthquakes = pd.read_csv('../Data/Clean/earthquakes_clean.csv')
volcanoes = pd.read_csv('../Data/Clean/volcanoes_clean.csv')

In [ ]:
# Filters

earthquakes = earthquakes[earthquakes.Year>=minYear]
earthquakes = earthquakes[earthquakes.Magnitude>=minMagnitude]

volcanoes = volcanoes[volcanoes.Year>=minYear]


In [ ]:
print('--Earthquakes--')
earthquakes

In [ ]:
print('--Volcanoes--')
volcanoes

In [ ]:
def catalog_to_dataframe(cat):
    times = []
    lats = []
    lons = []
    depths = []
    magnitudes = []
    magnitudestype = []
    for event in cat:
        if len(event.origins) != 0 and len(event.magnitudes) != 0:
            times.append(UTCDateTime(event.origins[0].time.datetime))
            lats.append(event.origins[0].latitude)
            lons.append(event.origins[0].longitude)
            depths.append(event.origins[0].depth)
            magnitudes.append(event.magnitudes[0].mag)
    cat_df = pd.DataFrame({'Latitude':lats,'Longitude':lons,'Depth':depths,'Magnitude':magnitudes,'Times':times})
    return cat_df

In [ ]:
EventPopulations_f = '../Data/Processed/EventPopulations.npy'

# EDA for clustering potential
# ind = 0 #for testing
# n_events = []
# event_id = []
# for ind in range(earthquakes.shape[0]):
#     try:
#         Event = earthquakes.iloc[ind]
#         DateString = UTCDateTime(str(pd.to_datetime(str(Event.Day) + '/' + str(Event.Month) + '/' + str(Event.Year) + ' ' + str(Event.Time),dayfirst=True)))
#         Start = DateString - HoursWindow_before*60*60
#         End = DateString + HoursWindow_after*60*60
#         client = Client("IRIS")
#         cat = client.get_events(starttime=Start,endtime=End,latitude=Event.Latitude,longitude=Event.Longitude,maxradius=maxRadius,minmagnitude=minMagnitude_query,maxdepth=Event.Depth + 10000)
#         cat_df = catalog_to_dataframe(cat)
#         n_events.append(cat_df.shape[0])
#         event_id.append(Event.ID)
#         time.sleep(0.15)
#         print('Query Progress: ' + str((ind/earthquakes.shape[0])*100) + '%')
#     except:
#         print('----Server Exception----')
#         continue
# na_events = np.array(n_events.copy())
# aevent_id = np.array(event_id.copy())
# EventPopulations = np.array([aevent_id.reshape(-1,1),na_events.reshape(-1,1)]).reshape(2,-1).T
# np.save(EventPopulations_f,EventPopulations)
EventPopulations = np.load(EventPopulations_f)
x_id = EventPopulations[:,0]
y_n = EventPopulations[:,1]


In [ ]:
plt.figure(figsize=(20,5))
plt.xlabel('Event ID',fontweight='bold',fontsize=20)
plt.ylabel('N',fontweight='bold',fontsize=20)
plt.title('N Contemporaneous Events',fontweight='bold',fontsize=20)
plt.bar(x_id,y_n)
plt.figure(figsize=(10,10))
plt.hist(y_n)
plt.yscale('log')
plt.ylabel('Count',fontweight='bold',fontsize=20)
plt.xlabel('N',fontweight='bold',fontsize=20)
plt.title('Distribution',fontweight='bold',fontsize=20)
plt.text(np.round(np.mean(plt.gca().get_xlim())),0.5*np.round(np.mean(plt.gca().get_ylim())),' min: ' + str(np.round(np.min(y_n))) + '\n max: ' + str(np.round(np.max(y_n))) + '\n mean: ' + str(np.round(np.mean(y_n))) + '\n std: ' + str(np.round(np.std(y_n))),fontweight='bold',fontsize=20,verticalalignment='top',horizontalalignment='center')

Suggested_minEvent_Cull = np.ceil(np.mean(y_n) + np.std(y_n))
print('-\n Suggested minimum number of events to perform cluster analysis: ' + str(Suggested_minEvent_Cull) + '\n-')

In [ ]:

# Event_Minimum = 70
Event_Minimum = 100

Culled_Count = EventPopulations[:,1][EventPopulations[:,1]>Event_Minimum]
Culled_ID = EventPopulations[:,0][EventPopulations[:,1]>Event_Minimum]

earthquakes_culled = earthquakes.loc[earthquakes['ID'].isin(Culled_ID)]
earthquakes_culled
earthquakes_culled["Date_UTC"] = np.nan
earthquakes_culled["Event_Cloud"] = np.nan
for ind in range(earthquakes_culled.shape[0]):
    Event = earthquakes_culled.iloc[ind]
    DateString = UTCDateTime(str(pd.to_datetime(str(Event.Day) + '/' + str(Event.Month) + '/' + str(Event.Year) + ' ' + str(Event.Time),dayfirst=True)))
    earthquakes_culled.iloc[ind,earthquakes_culled.columns.get_loc('Date_UTC')] = DateString

In [ ]:
# Build Cluster Datasets (Event_Cloud)

for ind in range(earthquakes_culled.shape[0]):
    Event = earthquakes_culled.iloc[ind]
    DateString = UTCDateTime(str(pd.to_datetime(str(Event.Day) + '/' + str(Event.Month) + '/' + str(Event.Year) + ' ' + str(Event.Time),dayfirst=True)))
    Start = DateString - HoursWindow_before*60*60
    End = DateString + HoursWindow_after*60*60
    client = Client("IRIS")
    cat = client.get_events(starttime=Start,endtime=End,latitude=Event.Latitude,longitude=Event.Longitude,maxradius=maxRadius,minmagnitude=minMagnitude_query,maxdepth=Event.Depth + 10000)
    cat_df = catalog_to_dataframe(cat)
    earthquakes_culled.iat[ind,earthquakes_culled.columns.get_loc('Event_Cloud')] = cat_df
    time.sleep(0.15)
    print('Query Progress: ' + str((ind/earthquakes_culled.shape[0])*100) + '%')
    # except:
    #     print('----Server Exception----')
    #     continue

In [16]:
DataSet = earthquakes_culled.copy()
DataSet.to_pickle('../Data/Processed/Compiled_DataSet.pkl')